<a href="https://colab.research.google.com/github/Kacper-W-Kozdon/notebook-testing-ivy/blob/main/Sarcasm_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IVY TRANSPILER

Installing kaggle and uploading the API key necessary to use it.

In [1]:
!pip install -q kaggle
from google.colab import files
from google.colab import userdata
import os
files.upload(); #Upload kaggle.json - you can get from the kaggle account settings, from the API section.

Saving kaggle.json to kaggle.json


Installing packages necessary to use torch's transformers.

In [2]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 8.6 MB/s eta 0:00:00


To use the API, credentials need to be copied into the kaggle folder. If everything works, the output will show the list of available datasets.

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

ref                                                  title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
sudarshan24byte/online-food-dataset                  Online Food Dataset                                  3KB  2024-03-02 18:50:30           9758        179  0.9411765        
syedanwarafridi/vehicle-sales-data                   Vehicle Sales Data                                  19MB  2024-02-21 20:16:17          19316        338  1.0              
julianoorlandi/spotify-top-songs-and-audio-features  Spotify Top Songs and Audio Features               526KB  2024-03-18 12:49:50           1319         26  1.0              
bhavikjikadara/student-study-performance             Student Study Performance                            9KB  2024-03-0

Preparing the ivy library.

In [4]:
#Inser the correct user when cloning the repos. Make sure that they are up-to-date.

!git clone "https://github.com/Kacper-W-Kozdon/demos.git"
!git clone "https://github.com/Kacper-W-Kozdon/ivy.git"
!pip install -U -q accelerate>=0.21.0 mlflow datasets>=2.14.5 nlp 2>/dev/null
!cd "/content/ivy"
!pip install --user -e .

Cloning into 'demos'...
remote: Enumerating objects: 3152, done.
remote: Counting objects: 100% (719/719), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 3152 (delta 627), reused 537 (delta 537), pack-reused 2433
Receiving objects: 100% (3152/3152), 16.83 MiB | 24.83 MiB/s, done.
Resolving deltas: 100% (1821/1821), done.
Cloning into 'ivy'...
remote: Enumerating objects: 189066, done.
remote: Counting objects: 100% (864/864), done.
remote: Compressing objects: 100% (382/382), done.
remote: Total 189066 (delta 601), reused 635 (delta 482), pack-reused 188202
Receiving objects: 100% (189066/189066), 164.23 MiB | 10.88 MiB/s, done.
Resolving deltas: 100% (152217/152217), done.


Next: import the ivy library and get the dataset.

In [5]:
import ivy
!kaggle datasets download -d danofer/sarcasm
!cp -f sarcasm.zip '/content/demos/Contributor_demos/Sarcasm Detection/'
!unzip '/content/demos/Contributor_demos/Sarcasm Detection/sarcasm.zip' -d '/content/demos/Contributor_demos/Sarcasm Detection/'

 98% 211M/216M [00:02<00:00, 61.4MB/s]
100% 216M/216M [00:02<00:00, 85.4MB/s]
Archive:  /content/demos/Contributor_demos/Sarcasm Detection/sarcasm.zip
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/test-balanced.csv  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/test-unbalanced.csv  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/train-balanced-sarc.csv.gz  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/train-balanced-sarcasm.csv  


Import the libraries suggested in the model which is to be transpiled.

In [8]:
# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
import gc  # For garbage collection to manage memory
import re  # For regular expressions
import numpy as np  # For numerical operations and arrays

# Libraries to accompany torch's transformers
import tqdm
import boto3
import requests
import regex
import sentencepiece
import sacremoses

import warnings  # For handling warnings
warnings.filterwarnings("ignore")  # Ignore warning messages

import tensorflow as tf
import torch  # PyTorch library for deep learning
from transformers import AutoModel, AutoTokenizer  # Transformers library for natural language processing
from transformers import TextDataset, LineByLineTextDataset, DataCollatorForLanguageModeling, \
pipeline, Trainer, TrainingArguments, DataCollatorWithPadding  # Transformers components for text processing
from transformers import AutoModelForSequenceClassification  # Transformer model for sequence classification

import accelerate

from nlp import Dataset  # Import custom 'Dataset' class for natural language processing tasks
from imblearn.over_sampling import RandomOverSampler  # For oversampling to handle class imbalance
import datasets  # Import datasets library
from datasets import Dataset, Image, ClassLabel  # Import custom 'Dataset', 'ClassLabel', and 'Image' classes
from transformers import pipeline  # Transformers library for pipelines
from bs4 import BeautifulSoup  # For parsing HTML content

import matplotlib.pyplot as plt  # For data visualization
import itertools  # For working with iterators
from sklearn.metrics import (  # Import various metrics from scikit-learn
    accuracy_score,  # For calculating accuracy
    roc_auc_score,  # For ROC AUC score
    confusion_matrix,  # For confusion matrix
    classification_report,  # For classification report
    f1_score  # For F1 score
)

from datasets import load_metric  # Import load_metric function to load evaluation metrics

from tqdm import tqdm  # For displaying progress bars
tqdm.pandas()  # Enable progress bars for pandas operations

Set the seeds.

In [9]:
tf.keras.utils.set_random_seed(0)
torch.manual_seed(0)

Get the API key for ivy transpiler from your account and upload it to the project. Move it to the correct directory.

In [10]:
files.upload(); #Upload key.pem - you can get from the kaggle account settings, from the API section.

Saving key.pem to key.pem


In [11]:
!mkdir ~/.ivy #It might be necessary to change ".ivy" to "ivy".
!cp key.pem ~/.ivy/

First we're loading the tokenizer and the model from torch. All of the basic set-up instructions can be found here: https://colab.research.google.com/github/pytorch/pytorch.github.io/blob/master/assets/hub/huggingface_pytorch-transformers.ipynb#scrollTo=72d8f2de

In [12]:
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased')
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-cased')

masked_lm_model = torch.hub.load('huggingface/pytorch-transformers', 'modelForMaskedLM', 'bert-base-cased')

Downloading: "https://github.com/huggingface/pytorch-transformers/zipball/main" to /root/.cache/torch/hub/main.zip


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
from ivy.stateful.module import Module
from ivy.stateful.sequential import Sequential
from ivy.stateful.layers import *
from ivy.stateful.losses import *
from ivy.stateful.optimizers import *
from ivy.stateful.activations import *
from ivy.stateful.initializers import *
from ivy.stateful.norms import *

In [15]:
ivy.set_backend("tensorflow")
tokenizer_tf = ivy.transpile(tokenizer, source="torch", to="tensorflow")
model_tf = ivy.transpile(model, source="torch", to="tensorflow")
masked_lm_model_tf = ivy.transpile(masked_lm_model, source="torch", to="tensorflow")

In [23]:
df = pd.read_csv("/content/demos/Contributor_demos/Sarcasm Detection/train-balanced-sarcasm.csv")
df = df.drop_duplicates()
df = df.rename(columns={'comment': 'title'})
df = df[['label', 'title']]
df = df[~df['label'].isnull()]
df = df[~df['title'].isnull()]
df.sample(5)

,label,title
797130,0,Routine?
464103,0,I think Apollos will be a good fit for the team.
96237,0,It costs 400 000 000$ to fire this weapon for ...
598115,0,found the city slicker.
217679,0,"If you think regular St Marys is bad, take a l..."
